<img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/>

# <center>Getting Started with the Arize Platform</center>
## <center>Investigating Embedding Drift in Image Classification</center>

**In this walkthrough, we are going to ingest embedding data and look at embedding drift.** 

In this scenario, you are in charge of maintaining an Image Classification model. Your model, resnet-50, will classify the input images into the 10 predefined categories of the Fashion MNIST (see [dataset](https://huggingface.co/datasets/arize-ai/fashion_mnist_quality_drift)). However, once the model is released into production, you notice that the performance of the model has degraded over a period of time.


This notebook will show you how Arize can automatically surface and troubleshoot the reason for this performance degradation by analyzing _image vectors_ associated with the input images so that you can take the right action to retrain your model/clean your data, saving you time and effort to correctly wrangle the datasets and visualize them. In this example, there are worse quality images in the production set during some period of time.

It is worth noting that, according to our research, inspecting embedding drift can surface problems with your data before they cause performance degradation.

In this tutorial, we will start from scratch. We will:
* Download the features, predictions and actuals of a dataset we have curated for this tutorial
* Automatically generate embeddings using the Arize SDK
* Log the inferences into Arize
* Visually explore embeddings in the Arize Platform

**Note**: This example compares training vs production data. Arize supports sending only one dataset.

Let's get started! 

# Step 0. Install Dependencies, Import Libraries, Use GPU 📚

To have automatic embedding generation functionality from the Arize SDK, we need to specify the
extra `[AutoEmbeddings]`.

⚠️ Use a GPU to save time generating embeddings. Click on 'Runtime', select 'Change Runtime Type' and
select 'GPU'.

In [ ]:
!pip install -q 'arize[AutoEmbeddings]'

import os
import uuid
from datetime import datetime

import pandas as pd
from arize.pandas.logger import Client
from arize.utils.types import Environments, ModelTypes, EmbeddingColumnNames, Schema

# Step 1. Download the data

We have curated a dataset for you so that you can send it to Arize in this tutorial.


In [ ]:
url="https://storage.googleapis.com/arize-assets/fixtures/Embeddings/arize-demo-models-data/CV/human_actions_quality_drift"

train_df = pd.read_csv(url + "_training.csv")
prod_df = pd.read_csv(url + "_production.csv")

In [ ]:
train_df.head()

In the table above you can observe how we have a column for URLs, which will use this URL to render the images on the Arize Platform. However, to generate the embedding vectors, we need to load the images locally. To do so, we download the compressed images and extract them as follows:

In [ ]:
!wget "https://storage.googleapis.com/arize-assets/fixtures/Embeddings/CV-public-images/human-actions-quality-drift.zip"
!unzip -q "human-actions-quality-drift.zip"

Next, we need a column with the local path to these images. We have prepared this so we can simply map it from the URLs via the function `get_local_path_from_url`.

In [ ]:
def get_local_path_from_url(url):
    return url.split("CV-public-images/")[-1]

In [ ]:
train_df['local_path'] = train_df['url'].map(get_local_path_from_url)
prod_df['local_path'] = prod_df['url'].map(get_local_path_from_url)

Let's inspect our data. Now we have a column with the local path to the images, which will be used to load them and generate embeddings, and a column with URLs used by the Arize platform for rendering.

In [ ]:
train_df.head()

# Step 2. Generate embedding vectors using Arize

Arize offers the ability of generating embeddings seemlessly using large pre-trained models. In this scenario, we will use the pre-trained vision transformer `google/vit-base-patch16-224-in21k`.


**NOTE: We recommend utilizing GPUs to optimize embedding generation. In Google Colaboratory, navigate to the 'Runtime' menu and select 'Change runtime type'. If you are interested in accessing even more powerful GPUs, upgrade to Colab Pro for enhanced speed and performance.** 

The large vision transformer models that Arize's embedding generators use have already been trained in such a huge amount of data that the embeddings can capture relevant structure in your data without being fine-tuned.

First step is to import `EmbeddingGenerator` and `UseCases`.

In [ ]:
from arize.pandas.embeddings import EmbeddingGenerator, UseCases

To list the supported pre-trained models, run the following:

In [ ]:
EmbeddingGenerator.list_pretrained_models()

Next, we define our generator, choosing the model `google/vit-base-patch16-224-in21k`.

You can also set the `batch_size`. This allows you to process the data in smaller batches if you are running out of resources. The default `batch_size` is 100.

Arize then downloads the models and tokenizers from the 🤗 HuggingFace Hub.

In [ ]:
generator = EmbeddingGenerator.from_use_case(
    use_case=UseCases.CV.IMAGE_CLASSIFICATION,
    model_name="google/vit-base-patch16-224-in21k"
)

We can explore details about our generators as follows:

In [ ]:
generator

If we want to check details about our model or image processor, we can reference them as follows:

In [ ]:
generator.image_processor

To generate the embeddings, we must pass the dataframe and the name of the column in the dataframe that contains the path to the images

In [ ]:
train_df['image_vector'] = generator.generate_embeddings(local_image_path_col=train_df["local_path"])
prod_df['image_vector'] = generator.generate_embeddings(local_image_path_col=prod_df["local_path"])

Let's explore our dataframe and discover the new column `image_vector` with the generated embeddings!

In [ ]:
train_df.head()

# Step 3. Prepare your data to be sent to Arize


## Update the timestamps

The data that you are working with was constructed in April of 2022. Hence, we will update the timestamps so they are current at the time that you're sending data to Arize.

In [ ]:
last_ts = max(prod_df['prediction_ts'])
now_ts = datetime.timestamp(datetime.now())
delta_ts = now_ts - last_ts    

train_df['prediction_ts'] = (train_df['prediction_ts'] + delta_ts).astype(float)
prod_df['prediction_ts'] = (prod_df['prediction_ts'] + delta_ts).astype(float)

## Add prediction ids

The Arize platform uses prediction IDs to link a prediction to an actual. Visit the [Arize documentation](https://docs.arize.com/arize/data-ingestion/model-schema/5.-prediction-id?q=prediction_id) for more details.

You can generate prediction IDs as follows:

In [ ]:
def add_prediction_id(df):
    return [str(uuid.uuid4()) for _ in range(df.shape[0])]

In [ ]:
train_df['prediction_id'] = add_prediction_id(train_df)
prod_df['prediction_id'] = add_prediction_id(prod_df)

# Step 4. Sending Data into Arize 💫

## Import and Setup Arize Client

The first step is to setup the Arize client. After that we will log the data.

Copy the Arize `API_KEY` and `SPACE_KEY` from your Space Settings page (shown below) to the variables in the cell below. We will also be setting up some metadata to use across all logging.

<img src="https://storage.googleapis.com/arize-assets/fixtures/copy-keys.png" width="700">

In [ ]:
SPACE_KEY = "SPACE_KEY"
API_KEY = "API_KEY"
arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)
model_id = "demo-auto-embeddings-cv-image-classification"
model_version = "1.0"
model_type = ModelTypes.SCORE_CATEGORICAL
if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print("✅ Import and Setup Arize Client Done! Now we can start using Arize!")


Now that our Arize client is set up, let's go ahead and log all of our data to the platform. For more details on how **`arize.pandas.logger`** works, visit our documentation.

[![Buttons_OpenOrange.png](https://storage.googleapis.com/arize-assets/fixtures/Buttons_OpenOrange.png)](https://docs.arize.com/arize/sdks-and-integrations/python-sdk/arize.pandas)

## Define the Schema 

A Schema instance specifies the column names for corresponding data in the dataframe. While we could define different Schemas for training and production datasets, the dataframes have the same column names, so the Schema will be the same in this instance.

To ingest non-embedding features, it suffices to provide a list of column names that contain the features in our dataframe. Embedding features, however, are a little bit different.

Arize allows you to ingest not only the embedding vector but the raw data associated with that embedding, or a URL link to that raw data. Therefore, up to 3 columns can be associated with the same _embedding object_*. To be able to do this, Arize's SDK provides the `EmbeddingColumnNames` class, used below.


***NOTE**: This is how we refer to the 3 possible pieces of information that can be sent as embedding objects:
* Embedding `vector` (required)
* Embedding `data` (optional): raw text associated with the embedding vector
* Embedding `link_to_data` (optional): link to the data file (image, audio, ...) associated with the embedding vector



Learn more [here](https://docs.arize.com/arize/sending-data/model-schema-reference#8.-embedding-features-unstructured).

In [ ]:
embedding_features = {
    # Dictionary keys will be name of embedding feature in the app
    "ViT image embedding": EmbeddingColumnNames(
        vector_column_name="image_vector",  
        link_to_data_column_name="url"
        )
}
    

# Define a Schema() object for Arize to pick up data from the correct columns for logging
schema = Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="pred_label",
    actual_label_column_name="label",
    embedding_feature_column_names=embedding_features
)

## Log Training Data

**Note**: This example compares training vs production data. Arize supports sending only one dataset.

In [ ]:
# Logging Training DataFrame
response = arize_client.log(
    dataframe=train_df,
    model_id=model_id,
    model_version=model_version,
    model_type=model_type,
    environment=Environments.TRAINING,
    schema=schema
)


# If successful, the server will return a status_code of 200
if response.status_code != 200:
    print(f"❌ logging failed with response code {response.status_code}, {response.text}")
else:
    print(f"✅ You have successfully logged training set to Arize")


## Log Production Data

In [ ]:
# Logging Production DataFrame
response = arize_client.log(
    dataframe=prod_df,
    model_id=model_id,
    model_version=model_version,
    model_type=model_type,
    environment=Environments.PRODUCTION,
    schema=schema
)

if response.status_code != 200:
    print(f"❌ logging failed with response code {response.status_code}, {response.text}")
else:
    print(f"✅ You have successfully logged production set to Arize")

# Step 5. Confirm Data in Arize ✅
Note that the Arize platform takes about 15 minutes to index embedding data. While the model should appear immediately, the data will not show up until the indexing is complete. Feel free to head over to the **Data Ingestion** tab for your model to watch Arize work its magic!🔮

You will be able to see the predictions, actuals, and feature importances that have been sent in the last 30 minutes, last day or last week.

An example view of the Data Ingestion tab from a model, when data is sent continuously over 30 minutes, is shown in the image below.

<img src="https://storage.googleapis.com/arize-assets/fixtures/data-ingestion-tab.png" width="700">

## Check the Embedding Data in Arize
Now, you can see how Arize surfaces the low quality images before your customer does and troubleshoots the degradation in performance to save you the time and effort. 

First, set the baseline to the training set that we logged before.

<img src="https://storage.googleapis.com/arize-assets/fixtures/Embeddings/CV/fashion_mnist_baseline_setup.gif" width="700">


Since your model contains embedding data, you will see it in your Model's Overview page. 

<img src="https://storage.googleapis.com/arize-assets/fixtures/Embeddings/CV/fashion_mnist_embedding.png" width="700">

 Click on the Embedding Name or the Euclidean Distance value to see how your embedding data is drifting over time. In the picture below we represent the global euclidean distance between your production set (at different points in time) and the baseline (which we set to be our training set). We can see there is a period of a week where suddenly the distance is remarkably higher. This shows us that during that time image data was sent to our model that was different than what it was trained on. This is the period of time when the quality of some images is worse.
 
<img src="https://storage.googleapis.com/arize-assets/fixtures/Embeddings/CV/fashion_mnist_drift.png" width="700">

In addition to the drift tracking plot above, below you can find the UMAP visualization of your data, according to the point in time selected. Notice that the production data and our baseline (training) data are superimposed, which is indicative that the model is seeing data in production similar to the data it was trained on.

<img src="https://storage.googleapis.com/arize-assets/fixtures/Embeddings/CV/fashion_mnist_no_drift_umap.png" width="700">

For further inspection, you may select a 3D UMAP view and clicked _Explore UMAP_ to expand the view. With this view we can interact in 3D with our dataset. We can zoom, rotate, and drag so we can see the areas of our dataset that are most interesting to us. Check out the workflow below:

<img src="https://storage.googleapis.com/arize-assets/fixtures/Embeddings/CV/fashion-mnist-workflow.gif" width="700">

In the display above, Arize offers many coloring options:
1. By Dataset: You can see that the coloring has been made to distinguish production data vs baseline data (training in this example). This is specifically useful to detect drift. In this example, we can see that there is some production data far away from any training data, giving an indication of severe dataset drift. We can identify exactly what datapoints our baseline is missing so that we can re-train effectively.
2. By Prediction Label: This coloring option gives an insight on how is our model making decisions. Where are the different classes located in the space? Is the model predicting one class in regions where it should be predicting another?
3. By Actual Label: This coloring option is great if we want to identify labeling issues. For instance, if inside the orange cloud, we can see a points of other colors, it is a good idea to check and see if the labels are wrong. Further we can use the corrected labels for re-training. This separation is specially difficult when clusters are joined, since both the model and UMAP have trouble separating the data-points.
4. By Correctness: This coloring option offers a quick way of identifying where the bulk of your model's mistakes are placed, giving you an area to pay attention to. In this example, we can see the difference between the red and blue images and almost all the red images have significantly worse quality (e.g. they are rotated and blurred).
5. By Confusion Matrix: This coloring option allows you to select a `positive class` and color the data-points as `True Positives`, `True Negatives`, `False Positives`, `False Negatives`.
6. By Feature: You can identify areas of the space where your model might be underperforming and, by coloring the points by feature, identify patterns at feature level. In other words, you can identify a slice of your data sharing a common feature (or features) that are causing a problem.
7. By Prediction Score: You can identify areas where your model is more confident of its predictions and areas where your model struggled more to make a decision.

More coloring options will be added to help you understand and debug your model and dataset.

# Wrap Up 🎁
Congratulations, you've now sent your first machine learning embedding data to the Arize platform!!

Additionally, if you want to remove this example model from your account, just click **Models** -> **CV-demo-fashion-mnist-quality-drift** -> **config** -> **delete**

### Overview
Arize is an end-to-end ML observability and model monitoring platform. The platform is designed to help ML engineers and data science practitioners surface and fix issues with ML models in production faster with:
- Automated ML monitoring and model monitoring
- Workflows to troubleshoot model performance
- Real-time visualizations for model performance monitoring, data quality monitoring, and drift monitoring
- Model prediction cohort analysis
- Pre-deployment model validation
- Integrated model explainability

### Website
Visit Us At: https://arize.com/model-monitoring/

### Additional Resources
- [What is ML observability?](https://arize.com/what-is-ml-observability/)
- [Monitor Unstructured Data with Arize](https://arize.com/blog/monitor-unstructured-data-with-arize)
- [Getting Started With Embeddings Is Easier Than You Think](https://arize.com/blog/getting-started-with-embeddings-is-easier-than-you-think)
- [Playbook to model monitoring in production](https://arize.com/the-playbook-to-monitor-your-models-performance-in-production/)
- [Using statistical distance metrics for ML monitoring and observability](https://arize.com/using-statistical-distance-metrics-for-machine-learning-observability/)
<!-- - [ML infrastructure tools for data preparation](https://arize.com/ml-infrastructure-tools-for-data-preparation/) -->
- [ML infrastructure tools for model building](https://arize.com/ml-infrastructure-tools-for-model-building/)
- [ML infrastructure tools for production](https://arize.com/ml-infrastructure-tools-for-production-part-1/)
<!-- - [ML infrastructure tools for model deployment and model serving](https://arize.com/ml-infrastructure-tools-for-production-part-2-model-deployment-and-serving/) -->

- [ML infrastructure tools for ML monitoring and observability](https://arize.com/ml-infrastructure-tools-ml-observability/)

Visit the [Arize Blog](https://arize.com/blog) and [Resource Center](https://arize.com/resource-hub/) for more resources on ML observability and model monitoring.
